## Analysis:

Griffin High School is the top performing school out of the 15 schools in the data set with an overall passing % of 81.34%, whereas Hernandez High School was the worst performing school with an overall passing % of 66.36%.

We can see that more spending per student, doesn't translate to better results, as the 645-680 range has by far the worst overall passing % of 66.76%. The best range was the 585-630 range which has a 73.88% overall passing %.

Small schools (<1000 students) also seem to bring the most success with a 79.07% overall passing % and the larger the school, the lower the % with the worst overall % belonging to large schools (2000-5000 students) at 70.29%.

Independent schools were also far better than Government schools with results. Independent schools had a 76.97% overall passing % whereas Government schools only had a 70.70% overall passing %.

In [29]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete_df

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12,Thomas High School,51,48,14,Independent,1635,1043130
39166,39166,Dawn Bell,F,10,Thomas High School,81,89,14,Independent,1635,1043130
39167,39167,Rebecca Tanner,F,9,Thomas High School,99,99,14,Independent,1635,1043130
39168,39168,Desiree Kidd,F,10,Thomas High School,72,77,14,Independent,1635,1043130


## Local Government Area Summary

In [30]:
# Work out each series
total_schools = len(school_data_complete_df["school_name"].unique())
total_students = school_data_complete_df["student_name"].count()
total_budget = sum(school_data_complete_df["budget"].unique())
avg_maths = school_data_complete_df["maths_score"].mean()
avg_reading = school_data_complete_df["reading_score"].mean()
pass_math = school_data_complete_df.loc[school_data_complete_df["maths_score"] >= 50,["student_name"]]
perc_math = (len(pass_math)/total_students)*100
pass_reading = school_data_complete_df.loc[school_data_complete_df["reading_score"] >= 50,["student_name"]]
perc_reading = (len(pass_reading)/total_students)*100
pass_overall = school_data_complete_df.loc[(school_data_complete_df["maths_score"] >= 50) & (school_data_complete_df["reading_score"] >= 50),["student_name"]]
perc_overall = (len(pass_overall)/total_students)*100

# Create the data frame
area_summary_df = pd.DataFrame({
    "Total Schools": [total_schools],
    "Total Students": [total_students],
    "Total Budget": [total_budget],
    "Average Maths Score": [avg_maths],
    "Average Reading Score": [avg_reading],
    "% Passing Maths": [perc_math],
    "% Passing Reading": [perc_reading],
    "% Passing Overall": [perc_overall]
                           })

# Format each cell that needed formatting and present
area_summary_df["Total Students"] = area_summary_df["Total Students"].map("{:,}".format)
area_summary_df["Total Budget"] = area_summary_df["Total Budget"].map("${:,.2f}".format)
area_summary_df["Average Maths Score"] = area_summary_df["Average Maths Score"].map("{:.2f}".format)
area_summary_df["Average Reading Score"] = area_summary_df["Average Reading Score"].map("{:.2f}".format)
area_summary_df["% Passing Maths"] = area_summary_df["% Passing Maths"].map("{:.2f}%".format)
area_summary_df["% Passing Reading"] = area_summary_df["% Passing Reading"].map("{:.2f}%".format)
area_summary_df["% Passing Overall"] = area_summary_df["% Passing Overall"].map("{:.2f}%".format)
area_summary_df



,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
0,15,"39,170","$24,649,428.00",70.34,69.98,86.08%,84.43%,72.81%


## School Summary

In [31]:
# Remove unnecessary columns
school_data_refined_df = school_data_complete_df.drop(["Student ID", "gender", "School ID"], axis=1)

# Set up columns that need to be defined
school_data_refined_df["Per Student Budget"] = school_data_refined_df["budget"] / school_data_refined_df["size"]
school_data_refined_df["% Passing Maths"] = school_data_refined_df["maths_score"] >= 50
school_data_refined_df["% Passing Reading"] = school_data_refined_df["reading_score"] >= 50
school_data_refined_df["% Passing Overall"] = (school_data_refined_df["maths_score"] >= 50) & (school_data_refined_df["reading_score"] >= 50)

# Rename columns and set School Name as Index
per_school_summary = school_data_refined_df.rename(columns={"school_name":"School Name"})
per_school_summary = per_school_summary.set_index(["School Name"])
organised_per_school_summary = per_school_summary[["type","student_name","budget","Per Student Budget","maths_score","reading_score","% Passing Maths","% Passing Reading","% Passing Overall"]]
renamed_per_school_summary = organised_per_school_summary.rename(columns={
    "type":"School Type",
    "student_name":"Total Students",
    "budget":"Total School Budget",
    "maths_score":"Average Maths Score",
    "reading_score":"Average Reading Score"
})

# Use groupby functions to get results for each school
type_summary = renamed_per_school_summary.groupby("School Name")["School Type"].first()
students_summary = renamed_per_school_summary.groupby("School Name")["Total Students"].count()
total_budget_summary = renamed_per_school_summary.groupby("School Name")["Total School Budget"].unique()
per_student_budget_summary = renamed_per_school_summary.groupby("School Name")["Per Student Budget"].mean()
avg_maths_summary = renamed_per_school_summary.groupby("School Name")["Average Maths Score"].mean()
avg_reading_summary = renamed_per_school_summary.groupby("School Name")["Average Reading Score"].mean()
pass_maths_summary = renamed_per_school_summary.loc[renamed_per_school_summary["% Passing Maths"] == True,:]
perc_maths_summary = (pass_maths_summary.groupby("School Name")["% Passing Maths"].count() / students_summary) * 100
perc_maths_summary = perc_maths_summary.rename("% Passing Maths")
pass_reading_summary = renamed_per_school_summary.loc[renamed_per_school_summary["% Passing Reading"] == True,:]
perc_reading_summary = (pass_reading_summary.groupby("School Name")["% Passing Reading"].count() / students_summary) * 100
perc_reading_summary = perc_reading_summary.rename("% Passing Reading")
pass_overall_summary = renamed_per_school_summary.loc[renamed_per_school_summary["% Passing Overall"] == True,:]
perc_overall_summary = (pass_overall_summary.groupby("School Name")["% Passing Overall"].count() / students_summary) * 100
perc_overall_summary = perc_overall_summary.rename("% Passing Overall")

# Merge all results together
per_summary_df1 = pd.merge(type_summary, students_summary, how="inner", on=["School Name"])
per_summary_df2 = pd.merge(per_summary_df1, total_budget_summary, how="inner", on=["School Name"])
per_summary_df3 = pd.merge(per_summary_df2, per_student_budget_summary, how="inner", on=["School Name"])
per_summary_df4 = pd.merge(per_summary_df3, avg_maths_summary, how="inner", on=["School Name"])
per_summary_df5 = pd.merge(per_summary_df4, avg_reading_summary, how="inner", on=["School Name"])
per_summary_df6 = pd.merge(per_summary_df5, perc_maths_summary, how="inner", on=["School Name"])
per_summary_df7 = pd.merge(per_summary_df6, perc_reading_summary, how="inner", on=["School Name"])
per_summary_df8 = pd.merge(per_summary_df7, perc_overall_summary, how="inner", on=["School Name"])

# Convert a couple of dtypes
per_school_summary = per_summary_df8.astype({"Total Students": "int64",
                        "Total School Budget": "float"})

per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Bailey High School,Government,4976,3124928.0,628.0,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,1081356.0,582.0,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,1884411.0,639.0,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,1763916.0,644.0,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,917500.0,625.0,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,3022020.0,652.0,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,248087.0,581.0,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,1910635.0,655.0,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,3094650.0,650.0,68.843100,69.039277,82.062592,81.978576,67.191766


In [32]:
# Confirm dtypes are correct
per_school_summary.dtypes

School Type               object
Total Students             int64
Total School Budget      float64
Per Student Budget       float64
Average Maths Score      float64
Average Reading Score    float64
% Passing Maths          float64
% Passing Reading        float64
% Passing Overall        float64
dtype: object

In [33]:
# Amend formatting
per_school_summary["Total Students"] = per_school_summary["Total Students"].map("{:,}".format)
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary["Average Maths Score"] = per_school_summary["Average Maths Score"].map("{:,.2f}".format)
per_school_summary["Average Reading Score"] = per_school_summary["Average Reading Score"].map("{:,.2f}".format)
per_school_summary["% Passing Maths"] = per_school_summary["% Passing Maths"].map("{:,.2f}%".format)
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].map("{:,.2f}%".format)
per_school_summary["% Passing Overall"] = per_school_summary["% Passing Overall"].map("{:,.2f}%".format)

# Display School Summary
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Bailey High School,Government,"4,976","$3,124,928.00",$628.00,72.35,71.01,91.64%,87.38%,80.08%
Cabrera High School,Independent,"1,858","$1,081,356.00",$582.00,71.66,71.36,90.85%,89.07%,80.79%
Figueroa High School,Government,"2,949","$1,884,411.00",$639.00,68.70,69.08,81.65%,82.81%,67.65%
Ford High School,Government,"2,739","$1,763,916.00",$644.00,69.09,69.57,82.44%,82.22%,67.47%
Griffin High School,Independent,"1,468","$917,500.00",$625.00,71.79,71.25,91.21%,88.49%,81.34%
Hernandez High School,Government,"4,635","$3,022,020.00",$652.00,68.87,69.19,80.95%,81.88%,66.36%
Holden High School,Independent,427,"$248,087.00",$581.00,72.58,71.66,89.93%,88.52%,78.92%
Huang High School,Government,"2,917","$1,910,635.00",$655.00,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,"4,761","$3,094,650.00",$650.00,68.84,69.04,82.06%,81.98%,67.19%


## Top Performing Schools (By % Overall Passing)

In [34]:
# Sort the schools in the per_school_summary DataFrame by % Overall Passing in descending order
top_schools = per_school_summary.sort_values("% Passing Overall", ascending=False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Griffin High School,Independent,"1,468","$917,500.00",$625.00,71.79,71.25,91.21%,88.49%,81.34%
Cabrera High School,Independent,"1,858","$1,081,356.00",$582.00,71.66,71.36,90.85%,89.07%,80.79%
Bailey High School,Government,"4,976","$3,124,928.00",$628.00,72.35,71.01,91.64%,87.38%,80.08%
Wright High School,Independent,"1,800","$1,049,400.00",$583.00,72.05,70.97,91.78%,86.67%,79.72%
Rodriguez High School,Government,"3,999","$2,547,363.00",$637.00,72.05,70.94,90.80%,87.40%,79.42%


## Bottom Performing Schools (By % Overall Passing)

In [35]:
# Sort the schools in the per_school_summary DataFrame by % Overall Passing in ascending order
bottom_schools = per_school_summary.sort_values("% Passing Overall", ascending=True)
bottom_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
School Name,,,,,,,,,
Hernandez High School,Government,"4,635","$3,022,020.00",$652.00,68.87,69.19,80.95%,81.88%,66.36%
Huang High School,Government,"2,917","$1,910,635.00",$655.00,68.94,68.91,81.69%,81.45%,66.71%
Johnson High School,Government,"4,761","$3,094,650.00",$650.00,68.84,69.04,82.06%,81.98%,67.19%
Wilson High School,Independent,"2,283","$1,319,574.00",$578.00,69.17,68.88,82.79%,81.30%,67.46%
Ford High School,Government,"2,739","$1,763,916.00",$644.00,69.09,69.57,82.44%,82.22%,67.47%


## Maths Scores by Year

In [36]:
# Create a table of the required columns
maths_year_df = school_data_complete_df.drop(["Student ID", "student_name", "gender", "reading_score", "School ID", "type", "size", "budget"], axis=1)
maths_year_df = maths_year_df.rename(columns={"school_name":"School Name"})
maths_year_df

,year,School Name,maths_score
0,9,Huang High School,94
1,12,Huang High School,43
2,12,Huang High School,76
3,12,Huang High School,86
4,9,Huang High School,69
...,...,...,...
39165,12,Thomas High School,48
39166,10,Thomas High School,89
39167,9,Thomas High School,99
39168,10,Thomas High School,77


In [37]:
# Creates series of school name and average score by year for the school using group by
maths_year_9 = maths_year_df.loc[maths_year_df["year"] == 9,["School Name","maths_score"]]
maths_year_9 = maths_year_9.groupby("School Name")["maths_score"].mean()
maths_year_9 = maths_year_9.rename("Year 9")
maths_year_10 = maths_year_df.loc[maths_year_df["year"] == 10,["School Name","maths_score"]]
maths_year_10 = maths_year_10.groupby("School Name")["maths_score"].mean()
maths_year_10 = maths_year_10.rename("Year 10")
maths_year_11 = maths_year_df.loc[maths_year_df["year"] == 11,["School Name","maths_score"]]
maths_year_11 = maths_year_11.groupby("School Name")["maths_score"].mean()
maths_year_11 = maths_year_11.rename("Year 11")
maths_year_12 = maths_year_df.loc[maths_year_df["year"] == 12,["School Name","maths_score"]]
maths_year_12 = maths_year_12.groupby("School Name")["maths_score"].mean()
maths_year_12 = maths_year_12.rename("Year 12")

# Merge all series
maths_scores_merge1 = pd.merge(maths_year_9, maths_year_10, how="inner", on=["School Name"])
maths_scores_merge2 = pd.merge(maths_scores_merge1, maths_year_11, how="inner", on=["School Name"])
maths_scores_by_year = pd.merge(maths_scores_merge2, maths_year_12, how="inner", on=["School Name"])

# Formatting
maths_scores_by_year["Year 9"] = maths_scores_by_year["Year 9"].map("{:,.2f}".format)
maths_scores_by_year["Year 10"] = maths_scores_by_year["Year 10"].map("{:,.2f}".format)
maths_scores_by_year["Year 11"] = maths_scores_by_year["Year 11"].map("{:,.2f}".format)
maths_scores_by_year["Year 12"] = maths_scores_by_year["Year 12"].map("{:,.2f}".format)
maths_scores_by_year

,Year 9,Year 10,Year 11,Year 12
School Name,,,,
Bailey High School,72.49,71.90,72.37,72.68
Cabrera High School,72.32,72.44,71.01,70.60
Figueroa High School,68.48,68.33,68.81,69.33
Ford High School,69.02,69.39,69.25,68.62
Griffin High School,72.79,71.09,71.69,71.47
Hernandez High School,68.59,68.87,69.15,68.99
Holden High School,70.54,75.11,71.64,73.41
Huang High School,69.08,68.53,69.43,68.64
Johnson High School,69.47,67.99,68.64,69.29


## Reading Score by Year

In [38]:
# Create a table of the required columns
reading_year_df = school_data_complete_df.drop(["Student ID", "student_name", "gender", "maths_score", "School ID", "type", "size", "budget"], axis=1)
reading_year_df = reading_year_df.rename(columns={"school_name":"School Name"})
reading_year_df

,year,School Name,reading_score
0,9,Huang High School,96
1,12,Huang High School,90
2,12,Huang High School,41
3,12,Huang High School,89
4,9,Huang High School,87
...,...,...,...
39165,12,Thomas High School,51
39166,10,Thomas High School,81
39167,9,Thomas High School,99
39168,10,Thomas High School,72


In [39]:
# Creates series of school name and average score by year for the school using group by
reading_year_9 = reading_year_df.loc[reading_year_df["year"] == 9,["School Name","reading_score"]]
reading_year_9 = reading_year_9.groupby("School Name")["reading_score"].mean()
reading_year_9 = reading_year_9.rename("Year 9")
reading_year_10 = reading_year_df.loc[reading_year_df["year"] == 10,["School Name","reading_score"]]
reading_year_10 = reading_year_10.groupby("School Name")["reading_score"].mean()
reading_year_10 = reading_year_10.rename("Year 10")
reading_year_11 = reading_year_df.loc[reading_year_df["year"] == 11,["School Name","reading_score"]]
reading_year_11 = reading_year_11.groupby("School Name")["reading_score"].mean()
reading_year_11 = reading_year_11.rename("Year 11")
reading_year_12 = reading_year_df.loc[reading_year_df["year"] == 12,["School Name","reading_score"]]
reading_year_12 = reading_year_12.groupby("School Name")["reading_score"].mean()
reading_year_12 = reading_year_12.rename("Year 12")

# Merge all series
reading_scores_merge1 = pd.merge(reading_year_9, reading_year_10, how="inner", on=["School Name"])
reading_scores_merge2 = pd.merge(reading_scores_merge1, reading_year_11, how="inner", on=["School Name"])
reading_scores_by_year = pd.merge(reading_scores_merge2, reading_year_12, how="inner", on=["School Name"])

# Formatting
reading_scores_by_year["Year 9"] = reading_scores_by_year["Year 9"].map("{:,.2f}".format)
reading_scores_by_year["Year 10"] = reading_scores_by_year["Year 10"].map("{:,.2f}".format)
reading_scores_by_year["Year 11"] = reading_scores_by_year["Year 11"].map("{:,.2f}".format)
reading_scores_by_year["Year 12"] = reading_scores_by_year["Year 12"].map("{:,.2f}".format)
reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
School Name,,,,
Bailey High School,70.90,70.85,70.32,72.20
Cabrera High School,71.17,71.33,71.20,71.86
Figueroa High School,70.26,67.68,69.15,69.08
Ford High School,69.62,68.99,70.74,68.85
Griffin High School,72.03,70.75,72.39,69.43
Hernandez High School,68.48,70.62,68.42,69.24
Holden High School,71.60,71.10,73.31,70.48
Huang High School,68.67,69.52,68.74,68.67
Johnson High School,68.72,69.30,69.97,67.99


## Scores by School Spending

In [40]:
# Remove unnecessary columns
spending_summary = per_summary_df8.drop(["School Type", "Total Students", "Total School Budget"], axis=1)

# Create bins and slice the data
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]
pd.cut(spending_summary["Per Student Budget"],spending_bins,labels=labels,include_lowest=True)
spending_summary["Spending Ranges (Per Student)"] = pd.cut(spending_summary["Per Student Budget"],spending_bins,labels=labels,include_lowest=True)

# Groupby the new column of the ranges to show the mean of the scores and then drop the original per student budget column
spending_summary = spending_summary.groupby("Spending Ranges (Per Student)").mean()
spending_summary = spending_summary.drop(["Per Student Budget"], axis=1)

# Formatting
spending_summary["Average Maths Score"] = spending_summary["Average Maths Score"].map("{:,.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].map("{:,.2f}".format)
spending_summary["% Passing Maths"] = spending_summary["% Passing Maths"].map("{:,.2f}%".format)
spending_summary["% Passing Reading"] = spending_summary["% Passing Reading"].map("{:,.2f}%".format)
spending_summary["% Passing Overall"] = spending_summary["% Passing Overall"].map("{:,.2f}%".format)

spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84%,86.39%,76.72%
$585-630,72.07,71.03,91.52%,87.29%,79.88%
$630-645,69.85,69.84,84.69%,83.76%,71.00%
$645-680,68.88,69.05,81.57%,81.77%,66.76%


## Scores by School Size

In [41]:
# Remove unnecessary columns and rename Total Students to Size
size_summary = per_summary_df8.drop(["School Type", "Per Student Budget", "Total School Budget"], axis=1)
size_summary = size_summary.rename(columns={"Total Students":"Size"})

# Create bins and slice the data
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
pd.cut(size_summary["Size"],size_bins,labels=labels,include_lowest=True)
size_summary["School Size"] = pd.cut(size_summary["Size"],size_bins,labels=labels,include_lowest=True)

# Groupby the new column of the ranges to show the mean of the scores and then drop the original Size column
size_summary = size_summary.groupby("School Size").mean()
size_summary = size_summary.drop(["Size"], axis=1)

# Formatting
size_summary["Average Maths Score"] = size_summary["Average Maths Score"].map("{:,.2f}".format)
size_summary["Average Reading Score"] = size_summary["Average Reading Score"].map("{:,.2f}".format)
size_summary["% Passing Maths"] = size_summary["% Passing Maths"].map("{:,.2f}%".format)
size_summary["% Passing Reading"] = size_summary["% Passing Reading"].map("{:,.2f}%".format)
size_summary["% Passing Overall"] = size_summary["% Passing Overall"].map("{:,.2f}%".format)

size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
School Size,,,,,
Small (<1000),72.34,71.64,90.81%,87.56%,79.07%
Medium (1000-2000),71.42,70.72,89.85%,86.71%,78.04%
Large (2000-5000),69.75,69.58,84.25%,83.30%,70.29%


## Scores by School Type

In [42]:
# Remove unnecessary columns
type_summary = per_summary_df8.drop(["Per Student Budget", "Total Students", "Total School Budget"], axis=1)

# Groupby the new column of the ranges to show the mean of the scores and then drop the original per student budget column
type_summary = type_summary.groupby("School Type").mean()

# Formatting
type_summary["Average Maths Score"] = type_summary["Average Maths Score"].map("{:,.2f}".format)
type_summary["Average Reading Score"] = type_summary["Average Reading Score"].map("{:,.2f}".format)
type_summary["% Passing Maths"] = type_summary["% Passing Maths"].map("{:,.2f}%".format)
type_summary["% Passing Reading"] = type_summary["% Passing Reading"].map("{:,.2f}%".format)
type_summary["% Passing Overall"] = type_summary["% Passing Overall"].map("{:,.2f}%".format)

type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Passing Overall
School Type,,,,,
Government,69.83,69.68,84.46%,83.59%,70.70%
Independent,71.37,70.72,89.20%,86.25%,76.97%
